# Distracted Driving Prediction
## Deep Learning with python, keras & tensorflow
### Sean O'Malley

The goal of this analysis is to identify the various types of distracted driving scenarios simply using images. We will accomplish this by using deep learning in python with the keras and a tensorflow backend. 

As a contextual reference, deep learning is a subset of machine learning that allows algorithms to train itself in order to perform tasks like image and speech recognition. Deep learning accomplishes this by revealing immense amounts of data to multi-layered neural networks.

This document will walk you through the entire process of: 
* Ingest a list of images
* Transform them into something that can be understood by a computer
* Split into test and training groups
* Prepare data to fit into a deep learning model


* Sequential API deep learning model
* Functional API deep learning model
* Functional API deep learning model
    + with inception module architecture
* Functional API deep learning model
    + with inception module architecture
    + using BatchNormalization to combat overfitting

Providing commentary, summary statistics and visualization along the way, we will determine the most successful model to move forward with in identifying distracted driving via images. 

We will conclude with a summary of the strengths, weaknesses and opportunities for improvement in the models to help fully understand the data, the models and the ultimate application of the analysis.


__Package Import__

In [57]:
# Basic Packages
import time
import numpy as np
import pandas as pd
import os, sys

# Deep Learning Packages
from keras.models import Model
from keras.layers import Input, Concatenate, Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import to_categorical
from keras import backend as K
from keras.optimizers import Adam, RMSprop

# Image Import & View Packages
import glob
import matplotlib.pyplot as plt
from PIL import Image


__Exploratory Data Analysis__

Ingest and view summaries of csv's to help us gain an understanding of the data we have. Looking at the data below, you'll see that we have 10 classes of images, living in separate photos. The photos are already categorized in their predefined classes by folders.

In [58]:
driver_imgs_list = pd.read_csv('driver_imgs_list.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [59]:
driver_imgs_list.head()

,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg
3,p002,c0,img_69092.jpg
4,p002,c0,img_92629.jpg


In [60]:
sample_submission.head()

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_1.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
1,img_10.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
2,img_100.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
3,img_1000.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
4,img_100000.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1


__Image Characteristics__

We see that the images are jpegs, 640x480 pixels and in full RGB color.

In [61]:
print(imagetest.format, imagetest.size, imagetest.mode)

JPEG (320, 240) RGB


__Manipulate and Transform Image Data__

The below outer for loop ingests each image from their respective folders while maintaining the classification of the image.

The nested for loop takes each image, converts it to black and white, reduces the image by a magnitued of 10, puts the image pixel values into a numpy array, appending values to the respective X (train) and y (test) lists.

In [62]:
X = []
y = []
for j in range(10):
    print('Load folder c{}'.format(j))
    path = os.path.join('imgs', 'train', 'c' + str(j), '*.jpg')
    img_list = glob.glob(path)
    for file in img_list:
        img = Image.open(file).convert('L')
        height, width = img.size
       # thumbnail is a in-place operation
        img = img.resize(( int(height/10), int(width/10) ), Image.ANTIALIAS) #, Image.ANTIALIAS
        pix = np.array(img.getdata()).reshape(img.size[0],img.size[1],1)
        X.append(pix)
        y.append(j)
        
    print("Number of train images: %s" % len(X))

Load folder c0
Number of train images: 2489
Load folder c1
Number of train images: 4756
Load folder c2
Number of train images: 7073
Load folder c3
Number of train images: 9419
Load folder c4
Number of train images: 11745
Load folder c5
Number of train images: 14057
Load folder c6
Number of train images: 16382
Load folder c7
Number of train images: 18384
Load folder c8
Number of train images: 20295
Load folder c9
Number of train images: 22424


__Encode X and y__

Because we are attempting to classify the images, we need to change y to categorical data for keras to understand what it is trying to classify. We also need to alter X to be a numpy array because the loop outputs a list.

In [69]:
Y = to_categorical(y)
print(Y.shape)

(22424, 10)


In [70]:
X = np.array(X) 
print(X.shape)

(22424, 64, 48, 1)


__Test / Train Shuffle and Split__

We then take the X and y lists and turn them into float value numpy arrays. After that, we shuffle the order of the data image by index and then split into test and train groups.

Prep Shuffle on index

In [71]:
ind = np.array(list(range(22424)))
np.random.shuffle(ind)
print(ind[:10])

[13852 10986 15934 16299 14507 14540   113  6070  5088  4286]


Apply shuffle to test and train

In [173]:
Xs = X[ind]
Ys = Y[ind]

__Determine validity of split__

Comparing X with Xs we can see that we have properly shuffled our test / train split.

In [174]:
print("shape of Ys:" + str(Ys.shape))
print("shape of Xs:" + str(Xs.shape))

shape of Ys:(22424, 10)
shape of Xs:(22424, 64, 48, 1)


Y looks properly shuffled

In [175]:
print(Y[0], Ys[0])

[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.] [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]


X also looks like things have been shuffled correctly

In [176]:
X[0].sum() - Xs[0].sum()

45537

__Normalization Process__

In [177]:
print(Xs.dtype)
print(Xs.max()) 

int64
255


In [178]:
Xs = (Xs-(Xs.max()/2))/Xs.max()

In [179]:
print(Xs.dtype)
print(Xs.mean())
print(Xs.max())
print(Xs.min())
print(Xs.nbytes/10**9) #size of the matrix in GB

float64
-0.142253596592
0.5
-0.5
0.551092224


In [180]:
# I want to save my disk space and change the dtype to float32
Xs = Xs.astype('float32')
print(Xs.dtype)
print(Xs.nbytes/10**9) # now I have half the size

float32
0.275546112


In [181]:
# now save to hdf5 (since it's huge I'd like to use hdf5)
# also hdf5 is a hierarchical data which lets you assign tags/groups or subgroups to your data
import h5py
with h5py.File('traindata.hdf5','w') as f:
    f.create_dataset('X', data=Xs)
    f.create_dataset('Y', data=Ys)

In [182]:
#reading the file
with h5py.File('traindata.hdf5','r') as f:
    X_train = f['X'][()] #the [()] means load all data
    # but you may load a part of data if you want for any reason
    X_part = f['X'][:100] #read first 100 samples

In [183]:
print(X_train.shape, X_part.shape)

(22424, 64, 48, 1) (100, 64, 48, 1)


In [ ]:
K.set_image_dim_ordering( 'tf' )

# fix random seed for reproducibility
seed = 123
numpy.random.seed(seed)

# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255

# one hot encode outputs
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
num_classes = y_test.shape[1]

In [ ]:
Xs = np.array(Xs, dtype = 'float32') 
Ys = np.array(Ys, dtype = 'float32') 

#plt.imshow(X[0])
#plt.imshow(Xs[0])
#X[0]